# Initialize Notebook

In [1]:
from collections import OrderedDict
from IPython.display import HTML,clear_output
import pandas as pd
import warnings
import json

from mstr_robotics.read_out_prj_obj import read_report
from mstr_robotics.prepare_AI_data import map_objects
from mstr_robotics.mstr_classes import mstr_global,md_searches

from mstr_robotics.read_out_prj_obj import io_attributes,read_prompts,read_cube,read_schema
from mstr_robotics._helper import msic
from mstr_robotics._connectors import mstr_api
from mstr_robotics.report import rep,prompts,cube

from mstr_robotics.dossier import doss_read_out,doss_read_out_det
from mstrio.connection import Connection

from mstr_robotics.mstr_pandas import df_helper

warnings.filterwarnings("ignore")
i_mstr_global=mstr_global()
i_read_report=read_report()
i_read_schema=read_schema()
i_rep=rep()
i_cube=cube()
i_read_cube=read_cube()
i_mstr_global=mstr_global()
i_md_searches=md_searches()
i_msic=msic()
io_att=io_attributes()
i_read_prompts=read_prompts()
i_mstr_api=mstr_api()
i_prompts=prompts()
i_df_helper=df_helper()
i_read_out_cube_att=read_cube()
i_map_objects=map_objects()
i_doss_read_out=doss_read_out()
i_doss_read_out_det=doss_read_out_det()
i_df_helper=df_helper()

## Config your project read out

In [2]:
user_path="..\\config\\user_d.json"
with open(user_path, 'r') as file:
    user_d = json.load(file)

jupyter_path="..\\config\\jupyter_objects_d.json"
with open(jupyter_path, 'r') as file:
    jupyter_objects_d = json.load(file)

conn_params =  user_d["conn_params"]
#GUID of MSTR Project
RAG_Process="turtorial_RAG"
project_id=jupyter_objects_d[RAG_Process]["project_id"]          #Tutorial project

#Guid for generated cubes
read_out_cbe_fold_id=jupyter_objects_d[RAG_Process]["read_out_cbe_fold_id"] 

#Guid from folder with cubes with attribute elements
ele_cbe_folder_id=jupyter_objects_d[RAG_Process]["ele_cbe_folder_id"] 

# Short cut folder for Reports and dossiers reachable for AI
read_out_ai_obj_fold_id=jupyter_objects_d[RAG_Process]["read_out_ai_obj_fold_id"] 

cube_RAG_form_val_ans_name=jupyter_objects_d[RAG_Process]["cube_RAG_form_val_ans_name"] 
cube_RAG_form_val_ans_id=jupyter_objects_d[RAG_Process]["cube_RAG_form_val_ans_id"] 
#cube_RAG_form_val_ans_id=None

cube_obj_prp_rel_name=jupyter_objects_d[RAG_Process]["cube_obj_prp_rel_name"] 
cube_obj_prp_rel_id=jupyter_objects_d[RAG_Process]["cube_obj_prp_rel_id"] 


cube_att_form_def_name=jupyter_objects_d[RAG_Process]["cube_att_form_def_name"] 
cube_att_form_def_id= jupyter_objects_d[RAG_Process]["cube_att_form_def_id"] 
#cube_att_form_def_id=None

cbe_doss_vis_def_detail_id=jupyter_objects_d[RAG_Process]["cbe_doss_vis_def_detail_id"] 
cbe_doss_vis_def_detail_name=jupyter_objects_d[RAG_Process]["cbe_doss_vis_def_detail_name"] 

cbe_cube_read_out_id=jupyter_objects_d[RAG_Process]["cbe_cube_read_out_id"] 
cbe_cube_read_out_name=jupyter_objects_d[RAG_Process]["cbe_cube_read_out_name"] 
cbe_cube_read_out_id=None

cbe_rep_def_id= jupyter_objects_d[RAG_Process]["cbe_rep_def_id"] 
cbe_rep_def_name=jupyter_objects_d[RAG_Process]["cbe_rep_def_name"] 

cbe_tbl_att_fct_mapping_id= jupyter_objects_d[RAG_Process]["cbe_tbl_att_fct_mapping_id"] 
cbe_tbl_att_fct_mapping_name=jupyter_objects_d[RAG_Process]["cbe_tbl_att_fct_mapping_name"] 


#cbe_tbl_att_fct_mapping_id=None
#cbe_rep_def_id=None
#cube_att_form_def_id=None
#cube_att_form_def_id=None

## Connect to your project

In [3]:
#Open connection to MSTR
conn_params["project_id"]=project_id
conn = Connection(**conn_params)
conn.headers['Content-type'] = "application/json"
load_d={"base_url":conn.base_url,"project_name":conn.project_name,"username":conn.username}

Connection to MicroStrategy Intelligence Server has been established.


## Read out relevant cubes for master data , reports and dashboards for BI - Chat Bot

In [4]:
cube_list_l= i_mstr_global.get_folder_obj_l(conn=conn,folder_id=ele_cbe_folder_id)
AI_rep_obj_l= i_mstr_global.get_obj_from_sh_fold(conn=conn,folder_id=read_out_ai_obj_fold_id)

Folder object named: 'Master data' with ID: '2B028FBC466C3DC839E1E092FDF4CC56'
Folder object named: 'AI enabeling' with ID: 'D41FC55D4282E9A5C943D9A8670336CF'


In [5]:
# Reports & dossiers with AI access
dos_rep_prp_rel_d=i_map_objects.get_doss_rep_prp(conn,object_l=AI_rep_obj_l)
dos_rep_prp_rel_l=dos_rep_prp_rel_d["prp_rep_l"]
dos_rep_prp_rel_df=pd.DataFrame(dos_rep_prp_rel_l)

AI_dos_rep_df=pd.DataFrame(AI_rep_obj_l)
AI_dos_rep_df.rename(columns={'name': 'dos_rep_name'}, inplace=True)

dos_rep_prp_rel_df = pd.merge(dos_rep_prp_rel_df[["project_id","rep_dos_id", "type","sub_type","prompt_id"]],
                            AI_dos_rep_df[["project_id","id","dos_rep_name","type","type_bez","sub_type","sub_type_bez"]],
                          left_on=['rep_dos_id'],right_on=['id'] , how='inner')
dos_rep_prp_rel_df=i_df_helper.clean_double_col(df=dos_rep_prp_rel_df)
dos_rep_prp_rel_df=dos_rep_prp_rel_df.drop("id", axis=1)
dos_rep_prp_rel_df=i_df_helper.add_prefix_col_to_df(df=dos_rep_prp_rel_df, prefix_col_d=load_d)

dos_rep_prp_rel_df=dos_rep_prp_rel_df.astype('str')

to_attribute=list(dos_rep_prp_rel_df.columns)

dos_rep_prp_rel_df["rep_prp_rel_dummy"]=1
dos_rep_prp_rel_d_l=[{"df":dos_rep_prp_rel_df,"tbl_name":"dos_rep_prp_rel_df", "update_policy":"Replace"}]
clear_output()

## load prompts and prompt objects relations

In [6]:
read_out_prp_all_d=i_read_prompts.run_read_out_prp_def(conn)
read_out_prp_all_df=pd.DataFrame(read_out_prp_all_d["prompt_def_l"])

obj_prp_rel_df=pd.DataFrame(read_out_prp_all_d["prompt_obj_l"])


obj_prp_rel_df = pd.merge(read_out_prp_all_df[["project_id","prompt_id", "prp_subType","promt_name"]],
                            obj_prp_rel_df[["project_id","prompt_id","object_id","obj_type","obj_prp_ans","key"]],
                          on=['prompt_id'] , how='left')
obj_prp_rel_df=i_df_helper.clean_double_col(df=obj_prp_rel_df)
obj_prp_rel_df=i_df_helper.add_prefix_col_to_df(df=obj_prp_rel_df, prefix_col_d=load_d)

to_attribute=list(obj_prp_rel_df.columns)
obj_prp_rel_df["obj_prp_rel_dummy"]=1

obj_prp_rel_d_l=[{"df":obj_prp_rel_df,"tbl_name":"obj_prp_rel_df", "update_policy":"Replace" }]
obj_prp_rel_d_l.extend(dos_rep_prp_rel_d_l)

i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=obj_prp_rel_d_l,mtdi_id=cube_obj_prp_rel_id,
                              cube_name=cube_obj_prp_rel_name,folder_id=read_out_cbe_fold_id,force=True)

allHierPrompt
SuperCube object named: 'obj_prp_rel' with ID: '1F0C564A40101C08ABF602AAB81B2324'


Uploading 2/2: 100%|██████████| 1/1 [00:00<00:00, 14.05it/s, rows=523]


Super cube 'obj_prp_rel' published successfully.


'1F0C564A40101C08ABF602AAB81B2324'

## prepare attribute expression prompts values

In [7]:
# read out all values for attribute forms
key_att_form_val_l= i_read_out_cube_att.read_cube_att_form_exp_val_l(conn,cube_list_l)
key_att_form_val_df=pd.DataFrame(key_att_form_val_l)


prompt_expression_df = obj_prp_rel_df[obj_prp_rel_df['prp_subType'].isin(['prompt_expression'])]

key_att_form_val_df = pd.merge(prompt_expression_df[["project_id","prompt_id", "prp_subType","promt_name","object_id"]],
                            key_att_form_val_df[["project_id","attribute_id","attribute_name","form_id","form_name","form_dataType","key","ele_prp_ans"]],
                          left_on=['object_id'],right_on=['attribute_id'] , how='inner')

key_att_form_val_df=i_df_helper.clean_double_col(df=key_att_form_val_df)
key_att_form_val_df = key_att_form_val_df.drop('object_id', axis=1)

key_att_form_val_df=i_df_helper.add_prefix_col_to_df(df=key_att_form_val_df, prefix_col_d=load_d)

key_att_form_val_df=key_att_form_val_df.astype('str')

key_att_form_val_df["key_att_form_val_dummy"]=1
key_att_form_val_d_l=[{"df":key_att_form_val_df,"tbl_name":"key_att_form_val_df", "update_policy":"Replace"}]

key_att_form_val_d_l.extend(dos_rep_prp_rel_d_l)
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=key_att_form_val_d_l,mtdi_id=cube_RAG_form_val_ans_id,
                              cube_name=cube_RAG_form_val_ans_name,folder_id=read_out_cbe_fold_id,force=True)

SuperCube object named: 'attribute_form_elements' with ID: 'B0A4AFCB4F8B53B4EB677D89B7A85CF0'


Uploading 2/2: 100%|██████████| 1/1 [00:00<00:00, 14.71it/s, rows=523]


Super cube 'attribute_form_elements' published successfully.


'B0A4AFCB4F8B53B4EB677D89B7A85CF0'

## read out table mappings

In [8]:
#read out the schema & builds a cube upload file for attributes, facts & table definitions

schema_mappings_d=i_read_schema.table_mappings(conn=conn,run_prop_d=load_d)

#better readabillity in the next cell
att_form_exp_df=schema_mappings_d["att_form_exp_df"]
fact_exp_df=schema_mappings_d["fact_exp_df"]
table_df=schema_mappings_d["table_df"]

tbl_att_fct_df = pd.merge(table_df[["project_id","table_id","table_name", "column_id","column_name","physicalTable_id","physicalTable_name"]],
                          att_form_exp_df[["project_id", 'table_id', "column_id", "attribute_id","attribute_name", "form_expressionId","form_name"]],
                          on=['project_id', 'table_id', 'column_id'], how='left')
i_df_helper.clean_double_col(df=tbl_att_fct_df)
tbl_att_fct_df = pd.merge(tbl_att_fct_df, 
                          fact_exp_df[["project_id", "column_id","fact_id","fact_name","fact_expressionId"]],
                          on=['project_id', 'column_id'], how='left')
i_df_helper.clean_double_col(df=tbl_att_fct_df)
tbl_att_fct_df["phys_tbl_col_id"]=tbl_att_fct_df["physicalTable_id"]+ "_"+ tbl_att_fct_df["column_id"]
tbl_att_fct_d_l=[{"df":tbl_att_fct_df,"tbl_name":"tbl_att_fct_df", "update_policy":"Replace"}]

i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=tbl_att_fct_d_l,mtdi_id=cbe_tbl_att_fct_mapping_id ,
                              cube_name=cbe_tbl_att_fct_mapping_name,folder_id=read_out_cbe_fold_id,force=True)


SuperCube object named: 'tbl_att_fct_mapping' with ID: 'D1D2CE9E43C95905EF6640AA0576FE1C'


Uploading 1/1: 100%|██████████| 1/1 [00:00<00:00,  8.06it/s, rows=2204]


Super cube 'tbl_att_fct_mapping' published successfully.


'D1D2CE9E43C95905EF6640AA0576FE1C'

## Read out all OLAP Cubes

In [9]:
cube_search_l=i_md_searches.search_for_type_l(conn=conn,obj_l=["776"])
cube_l = i_msic.get_key_form_dict_l(dict_l=cube_search_l,key="id")

cube_read_out_d_l=i_read_cube.read_all_olap_cubes(conn=conn,cube_l=cube_l)

cube_head_read_out_df=pd.DataFrame(cube_read_out_d_l["cube_head_d_l"])
cube_att_read_out_df=pd.DataFrame(cube_read_out_d_l["cube_att_d_l"])
cube_met_read_out_df=pd.DataFrame(cube_read_out_d_l["cube_met_d_l"])

cube_head_read_out_df=cube_head_read_out_df.astype('str')

cube_read_out_l_d=[{"df":cube_head_read_out_df,"tbl_name":"cube_head_read_out_df", "update_policy":"Replace"}]
cube_read_out_l_d.append({"df":cube_att_read_out_df,"tbl_name":"cube_att_read_out_df", "update_policy":"Replace"})
cube_read_out_l_d.append({"df":cube_met_read_out_df,"tbl_name":"cube_met_read_out_df", "update_policy":"Replace"})

i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=cube_read_out_l_d,mtdi_id=cbe_cube_read_out_id 
                              , cube_name=cbe_cube_read_out_name,folder_id=read_out_cbe_fold_id,force=True)


Created super cube 'cube_definitions' with ID: '628A864E44E5549D2AB6D28FD195A73A'.


Uploading 3/3: 100%|██████████| 1/1 [00:00<00:00, 22.48it/s, rows=183]


Super cube 'cube_definitions' published successfully.


'628A864E44E5549D2AB6D28FD195A73A'

## Read out Dossier

In [10]:
dossier_id_l=i_msic.get_obj_id_by_type_l(dict_l=AI_rep_obj_l, obj_type_l=[55])
#instance_id=i_mstr_api.create_dossier_instance(conn=conn, dossier_id=dossier_id)
doss_vis_d=i_doss_read_out_det.run_read_out_doss_hier_det(conn,dossier_id_l)


doss_vis_obj_df=pd.DataFrame(doss_vis_d)

doss_vis_obj_df=i_df_helper.add_prefix_col_to_df(df=doss_vis_obj_df, prefix_col_d=load_d)

doss_vis_obj_df.astype('str')

doss_vis_def_detail_l_d=[{"df":doss_vis_obj_df,"tbl_name":"doss_vis_obj_df", "update_policy":"Replace"}]
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=doss_vis_def_detail_l_d,mtdi_id=cbe_doss_vis_def_detail_id 
                              , cube_name=cbe_doss_vis_def_detail_name,folder_id=read_out_cbe_fold_id,force=True)


SuperCube object named: 'dashboard_definitions' with ID: '5FA79085461E723A503BAF8D3B4DE9D2'


Uploading 1/1: 100%|██████████| 1/1 [00:00<00:00, 14.67it/s, rows=460]


Super cube 'dashboard_definitions' published successfully.


'5FA79085461E723A503BAF8D3B4DE9D2'

## Read out Reports

In [11]:
report_id_l= i_msic.get_obj_id_by_type_l(dict_l=AI_rep_obj_l, obj_type_l=[3])
rep_def_d=i_read_report.read_reports(conn=conn,report_id_l=report_id_l,load_d=load_d)
#cbe_rep_def_id=None
rep_def_df=rep_def_d["rep_def_df"]
rep_def_d_l=[{"df":rep_def_df,"tbl_name":"rep_def_df", "update_policy":"Replace"}]
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=rep_def_d_l,mtdi_id=cbe_rep_def_id 
                              , cube_name=cbe_rep_def_name,folder_id=read_out_cbe_fold_id,force=True)

Error getting report DB8D5B064BBE3C24F541DAA81A507FDC definition. Check report ID.
I-Server Error ERR001, ()
Ticket ID: 121d9dc025324dc5a63ec4c2fb895d60
SuperCube object named: 'report_definitions' with ID: 'B7AE522248AD2604FA45489C1B750FBD'


Uploading 1/1: 100%|██████████| 1/1 [00:01<00:00,  1.41s/it, rows=24434]


Super cube 'report_definitions' published successfully.


'B7AE522248AD2604FA45489C1B750FBD'

### read out Attributeforms

In [12]:
#att_def_df=pd.DataFrame(attributes_l)

#remove table mappings to unique rows for each attribute form 
att_form_def_col=['project_id', 'attribute_id', 'attribute_name', 'form_id', 'form_name'
, 'form_category', 'form_type', 'displayFormat', 'form_dataType', 
'REST_form_type', 'form_precision', 'form_scale', 
'key_form', 'display_form_nr', 'browse_form_nr']
att_form_exp_df=att_form_exp_df[att_form_def_col].drop_duplicates()
att_form_exp_df=i_df_helper.add_prefix_col_to_df(df=att_form_exp_df, prefix_col_d=load_d)
att_form_exp_df=att_form_exp_df.astype('str')
att_form_exp_df.reset_index()
att_form_def_d_l=[{"df":att_form_exp_df,"tbl_name":"att_form_def_df", "update_policy":"Replace"}]
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=att_form_def_d_l,mtdi_id=cube_att_form_def_id,
                              cube_name=cube_att_form_def_name,folder_id=read_out_cbe_fold_id,force=True)

SuperCube object named: 'att_form_def' with ID: '870C97DA4303C51097C4068CE4DB6C4D'


Uploading 1/1: 100%|██████████| 1/1 [00:00<00:00, 16.13it/s, rows=178]


Super cube 'att_form_def' published successfully.


'870C97DA4303C51097C4068CE4DB6C4D'

In [13]:
#read out all attribute forms to support attribute form qualifications

#att_id_l=i_msic.get_key_form_dict_l(dict_l=att_l,key="id")
att_list_df=pd.concat([obj_prp_rel_df.loc[obj_prp_rel_df['obj_type'] == "attribute", ["object_id"]]
                       , doss_vis_obj_df.loc[doss_vis_obj_df['type'] == "attribute", ["object_id"]]
                       , rep_def_df.loc[rep_def_df['obj_type'] == "attribute", ["object_id"]]
                      ])

att_list_df.drop_duplicates()
att_id_l=att_list_df["object_id"].tolist()
#derived attributes are not supported in this version
#thus we need to filter them out
att_l=i_md_searches.search_for_type_l(conn=conn,obj_l=["12"] )

schema_att_id_l=i_msic.get_key_form_dict_l(dict_l=att_l,key="id")
read_out_att_id_l=i_msic.get_comon_val_l(list_1=att_id_l,list_2=schema_att_id_l)
read_out_att_id_l=i_msic.rem_dbl_in_l(read_out_att_id_l)
attributes_d=io_att.read_att_form_exp(conn=conn,att_id_l=read_out_att_id_l)
attributes_l=attributes_d["all_att_maps_l"]
att_def_df=pd.DataFrame(attributes_l)

#remove table mappings to unique rows for each attribute form 
att_form_def_col=['project_id', 'attribute_id', 'attribute_name', 'form_id', 'form_name'
, 'form_category', 'form_type', 'displayFormat', 'form_dataType', 
'REST_form_type', 'form_precision', 'form_scale', 
'key_form', 'display_form_nr', 'browse_form_nr']
att_form_def_df=att_def_df[att_form_def_col].drop_duplicates()
att_form_def_df=i_df_helper.add_prefix_col_to_df(df=att_form_def_df, prefix_col_d=load_d)
att_form_def_df=att_form_def_df.astype('str')
att_form_def_df.reset_index()
att_form_def_d_l=[{"df":att_form_def_df,"tbl_name":"att_form_def_df", "update_policy":"Replace"}]
i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=att_form_def_d_l,mtdi_id=cube_att_form_def_id,
                              cube_name=cube_att_form_def_name,folder_id=read_out_cbe_fold_id,force=True)

SuperCube object named: 'att_form_def' with ID: '870C97DA4303C51097C4068CE4DB6C4D'


Uploading 1/1: 100%|██████████| 1/1 [00:00<00:00, 16.57it/s, rows=175]


Super cube 'att_form_def' published successfully.


'870C97DA4303C51097C4068CE4DB6C4D'

In [14]:
print("HHHH")
clear_output()